# Finding baselines to beat with model
Used to predict next x months by just repeating observered volume.  
Measuring the model's effectiveness.
If the model can't beat these it's practically useless

https://towardsdatascience.com/how-to-build-a-baseline-model-be6ce42389fc

In [3]:
import numpy as np
import pandas as pd

import torch
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from pytorch_forecasting.data import GroupNormalizer
from sklearn import preprocessing

from ipynb.fs.full.training_preprocessing import GetDataset

In [1]:
from model_architecture import tft, training, val_dataloader, train_dataloader, validation, training
from sklearn.metrics import mean_absolute_error, mean_squared_error

Number of parameters in network: 105.6k


/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_forecasting/data/encoders.py:524: UserWarning: scale is below 1e-7 - consider not centering the data or using data with higher variance for numerical stability
  warnings.warn(
/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


In [4]:
df = GetDataset()
df

,Timestamp,subtotal,transaction_count,rain,temperature,holiday,oil_price,workforce_type_1,workforce_type_2,workforce_type_3,workforce_type_4,time_idx,constant_group,year,month,day,hour
0,17/05/2021 10:00,428.03,11.0,0.0,12.7,False,69.62,1.0,2.0,3.0,0.0,0,group_1,2021,5,17,10
1,17/05/2021 11:00,324.78,15.0,0.0,13.4,False,69.62,1.0,2.0,3.0,1.0,1,group_1,2021,5,17,11
2,17/05/2021 12:00,279.75,12.0,0.0,13.6,False,69.62,1.0,2.0,3.0,1.0,2,group_1,2021,5,17,12
3,17/05/2021 13:00,1630.59,29.0,0.0,14.1,False,69.62,1.0,2.0,3.0,1.0,3,group_1,2021,5,17,13
4,17/05/2021 14:00,1166.10,18.0,0.0,15.9,False,69.62,1.0,2.0,3.0,1.0,4,group_1,2021,5,17,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5062,30/11/2022 14:00,4779.11,82.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5062,group_1,2022,11,30,14
5063,30/11/2022 15:00,2460.84,54.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5063,group_1,2022,11,30,15
5064,30/11/2022 16:00,2026.30,43.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5064,group_1,2022,11,30,16
5065,30/11/2022 17:00,2346.13,44.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5065,group_1,2022,11,30,17


In [5]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)


# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 32  
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

NameError: name 'X_train' is not defined

In [6]:
# Given range, see how close the mean of previous 2 weeks and next 2 weeks is to the actual value
# Moving average

date_to_predict = pd.to_datetime('2022-10-20 10:00:00')


df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Get the transaction_count values from the last week of df
prev_week_1 = df[(df['Timestamp'] >= date_to_predict - pd.Timedelta(days=7)) & (df['Timestamp'] < date_to_predict)]['transaction_count']
prev_week_2 = df[(df['Timestamp'] >= date_to_predict - pd.Timedelta(days=15)) & (df['Timestamp'] < date_to_predict - pd.Timedelta(days=7))]['transaction_count']

next_week_1 = df[(df['Timestamp'] <= date_to_predict + pd.Timedelta(days=14)) & (df['Timestamp'] > date_to_predict + pd.Timedelta(days=7))]['transaction_count']
next_week_2 = df[(df['Timestamp'] <= date_to_predict + pd.Timedelta(days=21)) & (df['Timestamp'] > date_to_predict + pd.Timedelta(days=14))]['transaction_count']

actual_week = df[(df['Timestamp'] >= date_to_predict) & (df['Timestamp'] < date_to_predict + pd.Timedelta(days=7))]['transaction_count']

# For each field in last_2_weeks and next_2_weeks, get the mean between them 
# and store it in a new dataframe
predicted_with_means = []
for i in range (len(prev_week_1)):
    # Append to df2
    predicted_with_means.append((prev_week_1.iloc[i] + prev_week_2.iloc[i] + next_week_1.iloc[i] + next_week_2.iloc[i]) / 4)


# Get the mean absolute error between the actual values and the predicted values
mae = mean_absolute_error(actual_week, predicted_with_means)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 8.698412698412698


In [7]:
date_to_predict = pd.to_datetime('2022-10-20 10:00:00')

prev_date_1 = df[df['Timestamp'] == date_to_predict - pd.Timedelta(weeks=1)]['transaction_count'].values[0]
prev_date_2 = df[df['Timestamp'] == date_to_predict - pd.Timedelta(weeks=2)]['transaction_count'].values[0]

next_date_1 = df[df['Timestamp'] == date_to_predict + pd.Timedelta(weeks=1)]['transaction_count'].values[0]
next_date_2 = df[df['Timestamp'] == date_to_predict + pd.Timedelta(weeks=2)]['transaction_count'].values[0]

predicted = (prev_date_1 + prev_date_2 + next_date_1 + next_date_2) / 4

actual = df[df['Timestamp'] == date_to_predict]['transaction_count'].values[0]

print(f'Predicted: {predicted}, Actual: {actual}')
print(f'Difference: {abs(predicted - actual)}')

Predicted: 13.25, Actual: 11.0
Difference: 2.25


In [10]:
# Calculate the Mean Squared Error
mse = mean_squared_error(actual_week, predicted_with_means)

print(f'Mean Squared Error: {mse}')

Mean Squared Error: 137.77579365079364


In [19]:
# Calculate the seasonal naive baseline
seasonal_period = 9*365

seasonal_naive_forecast = df['transaction_count'].shift(seasonal_period)
seasonal_naive_forecast = seasonal_naive_forecast.iloc[-seasonal_period:]

# Drop the first n rows (where n is the seasonal period) since they don't have a seasonal naive forecast
ddf = df.iloc[seasonal_period:]

# Calculate the mean squared error (MSE) between the actual values and the seasonal naive forecast
#mse = ((df['transaction_count'] - df['seasonal_naive_forecast']) ** 2).mean()

mse = mean_squared_error(ddf, seasonal_naive_forecast)

print('Seasonal naive baseline MSE:', mse)

ValueError: Found input variables with inconsistent numbers of samples: [1782, 3285]